# 00 - Introduction

What is metaprogramming?

Wikipedia: A programming technique in which computer programs have the ability to treat other programs as their data.

It means that a program can be designed to read, generate, analyse, or transform other programs, and even modify itself while running.

The basic idea is we can use **code** to modify **code**.

Here are some examples:

- decorators: use a function to modify the behaviour of another function (typically).
- descriptors: use code to modify the behaviour of the dot (`.`) operator.
- metaclasses:
    - use code to create classes (types; same thing). Can be though of class (type) factory.
    - Superficially, metaclasses are not difficult to understand, but the details can get complicated.
    - Knowing when to use them is not easy. Generally speaking, they're useful for libraries/frameworks, not for application code.
    - This section will take lots of time to understand and absorb - re-reading will probably be necessary.

# 01 - Decorators and Descriptors - Review

Covered extensively in other summaries - no need to repeat here.

# 02 - The `__new__` Method

We've studied the `__init__` method quite a bit so far. It is basically a method that gets called right after the class instance has been created, usually invoked when we call the Class with arguments to instantiate an instance.

The `__new__` method is the method that is invoked to actually create the new object, as an instance of the desired class.

Since the `object` class provides a default implementation for `__new__` we rarely have to bother with it, but sometimes we want to intercept the instance creation to tweak things a bit.

The `__new__` method, unlike the `__init__` method is actually a **static** method, not an **instance** method. Which kinds of make sense since the instance does not exist yet - that's what the `__new__` method is trying to create.

Why it's not a **class** method is more complicated. We'll see why that's the case as we explore `__new__`.

Remember how we create instances of a class - we call the class with whatever arguments we need to initialize the class state:

```python
p = Person(name, age)
```

The creation of the class instance is then done in two steps:
1. The `__new__` method is called via inheritance from `object`. It receives, as arguments, the class object we want an instance of, and any additional arguments we pass to the creation call (e.g. `name` and `age`). It should return a new instance of the class (and it may have used the arguments to initialize stuff in the class too, that's up to how you write your `__new__` method)
3. If the object returned by `__new__` is an instance of the class specified in the call to `__new__`, then the `__init__` method is also called. The `__init__` method is an instance method and does not return anything (well, it returns None).

The manual approach to creating an instance of the class above is as follows:
```python
p = object.__new__(Person)
p.__init__(name, age)
```
There are some small differences between these two approaches which we'll cover soon.

The `__new__` method has the following parameters: `object.__new__(class, *args, **kwargs)`
- The `*args` and `**kwargs` are ignored, **but**, they must match what we intend to pass to the `__init__`of `class`
- A new object of type `class` is returned

##### Overriding the `__new__` method

We *can* override this method in our own custom classes. 

This should return a new object which is an instance of `class` but it doesn't *have to*, the consequence being that `__init__` is no longer called on the instance of `class`.

We don't typically override the internal implementations of `__new__`; rather, we do something before/after creating the object which we perform by delegating to `super().__new__` - similar to how we use decorators. 

We typically move some/all of the `__init__` logic into `__new__` after the instance has been instantiated because that's no different to running the logic in `__init__`.

Here's how we can do it in practice:

In [6]:
class Person:
    def __new__(cls, name, age):
        # do stuff here
        instance = super().__new__(cls)  
        # do stuff here
        return instance

When we call `Person('Guido', 68)`:
- Python calls `__new__(Person, 'Guido', 68)`
- `__new__` returns an object
    - if that object is of the same type as the "requested":
        - `new_object.__init__('Guido', 68)` is called
    - else:
        - don't attempt to call the `__init__`.

Note: 
- We don't need to pass the other *args/**kwargs to the `super().__new__()`, just the `Person.__new__()`. If we define an `__init__` for this `Person` class, we must have `name` and `age` as parameters.
- Using `object.__new__` instead of `super().__new__` wil raise issues when dealing with inheritance; child classes will be instantiated and initialised but parent classes will only be initialised, **not instantiated**.

#### Code

When we want to inherit from the builtin types such as `int`, `float`, etc., we can only override the `__new__`, not the `__init__` due to the implementation details in the C language.

So, if we wanted to create an integer using the `int()` constructor, but instead of just returning the integer, we return the square of it, we may try: 

In [3]:
class Squared(int):
    def __init__(self, x):
        super().__init__(x ** 2)

res = Squared(4)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

But the correct approach is:

In [5]:
class Squared(int):
    def __new__(cls, x):
        return super().__new__(cls, x ** 2)

Squared(4)

16

Here's a quick example:

Let's say we want to write a bound method that calculates the area. One (unconventional) way we could do this is: 

In [17]:
class Square:
    area = lambda self: self.w * self.l

    def __init__(self, w, l):
        self.w = w
        self.l = l

s = Square(3, 4)

s.area()

12

`area` is an honest-to-goodness bound method of `s`, which means the first argument is the instance `self`.

In [25]:
s.area

<bound method Square.<lambda> of <__main__.Square object at 0x000001C0B8207640>>

Another way we can do this is via `__new__`:

In [27]:
class Square:
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance

s = Square(3, 4)

s.area()

12

# 03 - How Classes are Created

# 04 - Inheriting from type

# 05 - Metaclasses

# 06 - Class Decorators

# 07 - Decorator Classes

# 08 - Metaclasses vs Class Decorators

# 09 - Metaclass Parameters

# 10 - The __prepare__ Method

# 11 - Metaprogramming Application 1

# 12 - Metaprogramming - Application 2

# 13 - Metaprogramming - Application 3

# 14 - Attribute Read Accessors

# 15 - Attribute Write Accessors

# 16 - Accessors - Application